# Stock Agent

In [45]:
# Basic Agent

In [46]:
import yfinance as yf
from langchain.agents import create_agent
from langchain.tools import tool

from dotenv import load_dotenv
import os

load_dotenv()

True

In [47]:
API_KEY = os.getenv("HUGGINGFACE_API_KEY")

## Tools

In [48]:
@tool
def get_stock_info(ticker: str) -> dict:
    """
    Fetch basic stock information using Yahoo Finance.

    Input:
        ticker (str): Stock ticker symbol, e.g. "AAPL" or "NVDA".

    Returns:
        dict with:
            - ticker
            - current_price
            - market_cap
            - trailing_pe
            - eps
            - dividend_yield
    """
    try:
        ticker = ticker.strip().upper()
        stock = yf.Ticker(ticker)
        info = stock.info  # <-- LIVE Yahoo Finance call

        if not info:
            return {"error": f"No data found for ticker '{ticker}'."}

        return {
            "ticker": ticker,
            "current_price": info.get("currentPrice"),
            "market_cap": info.get("marketCap"),
            "trailing_pe": info.get("trailingPE"),
            "eps": info.get("trailingEps"),
            "dividend_yield": info.get("dividendYield"),
        }

    except Exception as e:
        return {"error": str(e)}

## Agent

In [52]:
agent = create_agent(
    model="gpt-4o-mini",
    tools=[get_stock_info],
    system_prompt="You are a helpful assistant"
)

In [56]:
# Run the agent
agent.invoke(
    {"input": "hey"}
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}